In [1]:
import os
import pandas as pd
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from tqdm import tqdm
import json

In [2]:
TRAIN_PATH = '../dataset/train.tsv'
TEST_PATH = '../dataset/test.tsv'

In [3]:
df_train = pd.read_csv(TRAIN_PATH, sep = '\t')

In [4]:
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [5]:
df_test = pd.read_csv(TEST_PATH, sep = '\t')

In [6]:
df_test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [7]:
X_train = df_train.Phrase.values

In [8]:
Y_train = df_train.Sentiment.values

In [9]:
X_train

array(['A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .',
       'A series of escapades demonstrating the adage that what is good for the goose',
       'A series', ..., 'avuncular chortles', 'avuncular', 'chortles'],
      dtype=object)

In [10]:
Y_train

array([1, 2, 2, ..., 3, 2, 2], dtype=int64)

In [11]:
X_test = df_test.Phrase.values

In [12]:
X_test

array(['An intermittently pleasing but mostly routine effort .',
       'An intermittently pleasing but mostly routine effort', 'An', ...,
       'A long-winded ,', 'A long-winded', 'predictable scenario'],
      dtype=object)

In [13]:
X = np.concatenate((X_train, X_test))

In [14]:
X

array(['A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .',
       'A series of escapades demonstrating the adage that what is good for the goose',
       'A series', ..., 'A long-winded ,', 'A long-winded',
       'predictable scenario'], dtype=object)

In [15]:
X.shape, X_train.shape, X_test.shape

((222352,), (156060,), (66292,))

In [16]:
nlp = spacy.load("en")

In [17]:
[word.lemma_ for word in nlp(X[0].lower()) if word.lemma_ not in STOP_WORDS]

['series',
 'escapade',
 'demonstrate',
 'adage',
 'good',
 'goose',
 'good',
 'gander',
 ',',
 'occasionally',
 'amuse',
 'story',
 '.']

In [19]:
maxlen = 0
word_freq_dict = {}
runner = tqdm(X, total = len(X))
for sentence in runner:
    tokenized = nlp(sentence.lower())
    word_list = [str(word.lemma_) for word in tokenized if word.lemma_ not in STOP_WORDS]
    maxlen = max(maxlen, len(word_list))
    for word in word_list:
        if word not in word_freq_dict:
            word_freq_dict[word] = 0
        word_freq_dict[word] += 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 222352/222352 [21:41<00:00, 170.78it/s]


In [21]:
maxlen

35

In [22]:
word_freq_dict['.']

25972

In [23]:
word_freq_dict['!']

312

In [24]:
word_freq_dict[',']

58124

In [25]:
len(word_freq_dict)

15741

In [26]:
f = open("../dataset/word_freq_dict.json", "w")
json.dump(word_freq_dict, f)
f.close()

In [27]:
word_to_index = {}
for dex, word in enumerate(word_freq_dict):
    word_to_index[word] = dex

In [29]:
f = open("../dataset/word_to_index_train_and_test.json", "w")
json.dump(word_to_index, f)
f.close()